In [1]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning import Trainer


In [2]:
forecast_data = pd.read_csv("result_data/final_data_tft2.csv")
forecast_data = forecast_data.drop(columns=['Unnamed: 0'])
forecast_data = forecast_data.sort_values(['ss_id', 'pv_datetime'])
forecast_data['pv_datetime'] = pd.to_datetime(forecast_data['pv_datetime'])
forecast_data['date'] = forecast_data['pv_datetime'].dt.date
forecast_data['day_of_week'] = forecast_data['pv_datetime'].dt.dayofweek
forecast_data['month'] = forecast_data['pv_datetime'].dt.month


In [3]:
forecast_data.head()

,ss_id,init_time,step,generation,pv_datetime,pv_hour,hour,lat,long,tilt,...,sr,t2m,tcc,u10,u100,v10,v100,date,day_of_week,month
6293,3147,2019-03-17 00:00:00,0 days 18:00:00,1.069375e-10,2019-03-16 18:00:00,18,1,51.12,-0.02,40.0,...,21856324.0,279.14966,0.121735,5.416376,9.102982,-0.231537,-0.851471,2019-03-16,5,3
6294,3147,2019-03-17 00:00:00,0 days 19:00:00,0.000000e+00,2019-03-16 19:00:00,19,2,51.12,-0.02,40.0,...,21863492.0,277.78784,0.001434,4.447694,7.998607,0.227203,-0.140961,2019-03-16,5,3
6295,3147,2019-03-17 00:00:00,0 days 20:00:00,0.000000e+00,2019-03-16 20:00:00,20,3,51.12,-0.02,40.0,...,21863492.0,276.78490,0.441833,4.002581,7.498468,0.140137,-0.432358,2019-03-16,5,3
6296,3147,2019-03-17 00:00:00,0 days 21:00:00,0.000000e+00,2019-03-16 21:00:00,21,4,51.12,-0.02,40.0,...,21863492.0,276.85815,0.721863,3.675459,6.809860,-0.478973,-1.726532,2019-03-16,5,3
6297,3147,2019-03-17 00:00:00,0 days 22:00:00,0.000000e+00,2019-03-16 22:00:00,22,5,51.12,-0.02,40.0,...,21863492.0,277.41187,0.018034,3.383099,6.230782,-1.177109,-2.980850,2019-03-16,5,3


In [4]:
forecast_data.columns

Index(['ss_id', 'init_time', 'step', 'generation', 'pv_datetime', 'pv_hour',
       'hour', 'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf', 'duvrs',
       'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10',
       'v100', 'date', 'day_of_week', 'month'],
      dtype='object')

In [5]:
target_variable = 'generation'
static_features = ['ss_id', 'lat', 'long', 'tilt', 'orientation']
known_future_inputs = [
    'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 
    't2m', 'tcc', 'u10', 'u100', 'v10', 'v100', 'day_of_week', 'month', 'hour'
]

required_columns = static_features + known_future_inputs + [target_variable, 'pv_datetime', 'date']

forecast_data = forecast_data[required_columns]

forecast_data = forecast_data.fillna(method='bfill').fillna(method='ffill')



In [6]:
print(forecast_data.columns)

Index(['ss_id', 'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf',
       'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100',
       'v10', 'v100', 'day_of_week', 'month', 'hour', 'generation',
       'pv_datetime', 'date'],
      dtype='object')


In [7]:
forecast_data.head()

,ss_id,lat,long,tilt,orientation,dlwrf,dswrf,duvrs,hcc,lcc,...,u10,u100,v10,v100,day_of_week,month,hour,generation,pv_datetime,date
6293,3147,51.12,-0.02,40.0,180.0,17286896.0,14154521.0,1444081.8,0.0,0.103668,...,5.416376,9.102982,-0.231537,-0.851471,5,3,1,1.069375e-10,2019-03-16 18:00:00,2019-03-16
6294,3147,51.12,-0.02,40.0,180.0,18194266.0,14154777.0,1444113.8,0.0,0.001434,...,4.447694,7.998607,0.227203,-0.140961,5,3,2,0.000000e+00,2019-03-16 19:00:00,2019-03-16
6295,3147,51.12,-0.02,40.0,180.0,19073552.0,14154777.0,1444113.8,0.0,0.441833,...,4.002581,7.498468,0.140137,-0.432358,5,3,3,0.000000e+00,2019-03-16 20:00:00,2019-03-16
6296,3147,51.12,-0.02,40.0,180.0,20026864.0,14154777.0,1444113.8,0.0,0.721863,...,3.675459,6.809860,-0.478973,-1.726532,5,3,4,0.000000e+00,2019-03-16 21:00:00,2019-03-16
6297,3147,51.12,-0.02,40.0,180.0,21029888.0,14154777.0,1444113.8,0.0,0.018005,...,3.383099,6.230782,-1.177109,-2.980850,5,3,5,0.000000e+00,2019-03-16 22:00:00,2019-03-16


In [9]:
# forecast_data.to_csv("test2.csv")

In [ ]:
max_encoder_length = 48
max_prediction_length = 48